# Яндекс.Музыка

Сравнение Москвы и Петербурга окружено мифами. Например:
 * Москва — мегаполис, подчинённый жёсткому ритму рабочей недели;
 * Петербург — культурная столица, со своими вкусами.

На данных Яндекс.Музыки необходимо сравнить поведение пользователей двух столиц.

**Цель исследования** — проверить три гипотезы:
1. Активность пользователей зависит от дня недели. Причём в Москве и Петербурге это проявляется по-разному.
2. В понедельник утром в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города. 
3. Москва и Петербург предпочитают разные жанры музыки. В Москве чаще слушают поп-музыку, в Петербурге — русский рэп.

**Ход исследования**

Данные о поведении пользователей в файле `yandex_music_project.csv`. О качестве данных ничего не известно. Поэтому перед проверкой гипотез понадобится обзор данных. 

Таким образом, исследование необходимо выполнить в три этапа:
 1. Обзор данных.
 2. Предобработка данных.
 3. Проверка гипотез.



## Обзор данных

In [1]:
# импорт библиотеки pandas
import pandas as pd 

In [2]:
# чтение файла с данными и сохранение в df
df = pd.read_csv('yandex_music_project.csv') 

In [3]:
# получение первых 10 строк таблицы df 
display(df.head(10)) 

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday
5,842029A1,Преданная,IMPERVTOR,rusrap,Saint-Petersburg,13:09:41,Friday
6,4CB90AA5,True,Roman Messer,dance,Moscow,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Moscow,20:47:49,Wednesday
8,8FA1D3BE,И вновь продолжается бой,NaN,ruspop,Moscow,09:17:40,Friday
9,E772D5C0,Pessimist,NaN,dance,Saint-Petersburg,21:20:49,Wednesday


In [4]:
# получение общей информации о данных в таблице df
df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


Итак, в таблице семь столбцов. Тип данных во всех столбцах — `object`.

Согласно документации к данным:
* `userID` — идентификатор пользователя;
* `Track` — название трека;  
* `artist` — имя исполнителя;
* `genre` — название жанра;
* `City` — город пользователя;
* `time` — время начала прослушивания;
* `Day` — день недели.

В названиях колонок видны три нарушения стиля:
1. Строчные буквы сочетаются с прописными.
2. Встречаются пробелы.
3. "userID" лучше записать через "_"



Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.


**Выводы**

В каждой строке таблицы — данные о прослушанном треке. Часть колонок описывает саму композицию: название, исполнителя и жанр. Остальные данные рассказывают о пользователе: из какого он города, когда он слушал музыку. 

Предварительно можно утверждать, что, данных достаточно для проверки гипотез. Но встречаются пропуски в данных, а в названиях колонок — расхождения с хорошим стилем.

Нужно устранить проблемы в данных.

## Предобработка данных

### Стиль заголовков

In [5]:
# перечень названий столбцов таблицы df
df.columns 

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')

Переименование столбцов в соответствие с хорошим стилем:

Текущее название | Новое название
--- | ---
'  userID' | 'user_id'
'Track' | 'track'
'  City  ' | 'city'
'Day' | 'day'

In [6]:
# переименование столбцов в соответствие с хорошим стилем
df = df.rename(columns={'  userID': 'user_id', 
                        'Track': 'track', 
                        '  City  ': 'city', 
                        'Day': 'day'}) 

In [7]:
# проверка результатов
df.columns

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')

### Пропуски значений

In [8]:
# подсчёт пропусков
df.isna().sum() 

user_id       0
track      1231
artist     7203
genre      1198
city          0
time          0
day           0
dtype: int64

Не все пропущенные значения влияют на исследование. Так в `track` и `artist` пропуски не важны для текущей работы. Достаточно заменить их явными обозначениями.

Но пропуски в `genre` могут помешать сравнению музыкальных вкусов в Москве и Санкт-Петербурге. Было бы правильно установить причину пропусков и восстановить данные. В текущей работе такой возможности нет. Поэтому, придётся:
* заполнить эти пропуски явными обозначениями,
* оценить, насколько они повредят расчётам. 

Заменим пропущенные значения в столбцах `track`, `artist` и `genre` на строку `'unknown'`. Для этого создадим список `columns_to_replace`, переберем его элементы циклом `for` и для каждого столбца выполним замену пропущенных значений:

In [9]:
# замена пропущенных значений на 'unknown'
columns_to_replace = ['track', 'artist', 'genre']
df[columns_to_replace]=df[columns_to_replace].fillna('unknown')

In [10]:
# проверка, подсчёт пропусков
df.isna().sum() 

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64

### Дубликаты

In [11]:
# подсчёт явных дубликатов
df.duplicated().sum()

3826

In [12]:
# удаление явных дубликатов (с удалением старых индексов и формированием новых)
df = df.drop_duplicates().reset_index(drop = True)

In [13]:
# проверка на отсутствие дубликатов
df.duplicated().sum()

0

Дубликатов нет.

Теперь избавимся от неявных дубликатов в колонке `genre`. Например, название одного и того же жанра может быть записано немного по-разному. 

Выведем на экран список уникальных названий жанров, отсортированный в алфавитном порядке.

In [14]:
# Просмотр уникальных названий жанров
sorted(df['genre'].unique())

['acid',
 'acoustic',
 'action',
 'adult',
 'africa',
 'afrikaans',
 'alternative',
 'alternativepunk',
 'ambient',
 'americana',
 'animated',
 'anime',
 'arabesk',
 'arabic',
 'arena',
 'argentinetango',
 'art',
 'audiobook',
 'author',
 'avantgarde',
 'axé',
 'baile',
 'balkan',
 'beats',
 'bigroom',
 'black',
 'bluegrass',
 'blues',
 'bollywood',
 'bossa',
 'brazilian',
 'breakbeat',
 'breaks',
 'broadway',
 'cantautori',
 'cantopop',
 'canzone',
 'caribbean',
 'caucasian',
 'celtic',
 'chamber',
 'chanson',
 'children',
 'chill',
 'chinese',
 'choral',
 'christian',
 'christmas',
 'classical',
 'classicmetal',
 'club',
 'colombian',
 'comedy',
 'conjazz',
 'contemporary',
 'country',
 'cuban',
 'dance',
 'dancehall',
 'dancepop',
 'dark',
 'death',
 'deep',
 'deutschrock',
 'deutschspr',
 'dirty',
 'disco',
 'dnb',
 'documentary',
 'downbeat',
 'downtempo',
 'drum',
 'dub',
 'dubstep',
 'eastern',
 'easy',
 'electronic',
 'electropop',
 'emo',
 'entehno',
 'epicmetal',
 'estrada',


Видим неявные дубликаты названия `hiphop`:
* *hip*,
* *hop*,
* *hip-hop*.

Чтобы очистить от них таблицу, создадим функцию `replace_wrong_genres()` с двумя параметрами: 
* `wrong_genres` — список дубликатов,
* `correct_genre` — строка с правильным значением.

Функция исправит колонку `genre` в таблице `df`: заменит каждое значение из списка `wrong_genres` на значение из `correct_genre`.

In [15]:
# Функция для замены неявных дубликатов
def replace_wrong_genres(wrong_genres, correct_genres): 
    for wrong_genre in wrong_genres:
        df['genre'] = df['genre'].replace(wrong_genres, correct_genres)

In [16]:
# Устранение неявных дубликатов
duplicates = ['hip', 'hop', 'hip-hop'] # список исправляемых значений
new_name = 'hiphop' # правильное значение
replace_wrong_genres(duplicates, new_name)

Проверим, что заменили неправильные названия:

*   hip
*   hop
*   hip-hop

Выведем отсортированный список уникальных значений столбца `genre`:

In [17]:
# Проверка на неявные дубликаты
sorted(df['genre'].unique())

['acid',
 'acoustic',
 'action',
 'adult',
 'africa',
 'afrikaans',
 'alternative',
 'alternativepunk',
 'ambient',
 'americana',
 'animated',
 'anime',
 'arabesk',
 'arabic',
 'arena',
 'argentinetango',
 'art',
 'audiobook',
 'author',
 'avantgarde',
 'axé',
 'baile',
 'balkan',
 'beats',
 'bigroom',
 'black',
 'bluegrass',
 'blues',
 'bollywood',
 'bossa',
 'brazilian',
 'breakbeat',
 'breaks',
 'broadway',
 'cantautori',
 'cantopop',
 'canzone',
 'caribbean',
 'caucasian',
 'celtic',
 'chamber',
 'chanson',
 'children',
 'chill',
 'chinese',
 'choral',
 'christian',
 'christmas',
 'classical',
 'classicmetal',
 'club',
 'colombian',
 'comedy',
 'conjazz',
 'contemporary',
 'country',
 'cuban',
 'dance',
 'dancehall',
 'dancepop',
 'dark',
 'death',
 'deep',
 'deutschrock',
 'deutschspr',
 'dirty',
 'disco',
 'dnb',
 'documentary',
 'downbeat',
 'downtempo',
 'drum',
 'dub',
 'dubstep',
 'eastern',
 'easy',
 'electronic',
 'electropop',
 'emo',
 'entehno',
 'epicmetal',
 'estrada',


**Выводы**

Предобработка обнаружила три проблемы в данных:

- нарушения в стиле заголовков,
- пропущенные значения,
- дубликаты — явные и неявные.

Бали переименованы заголовки, удалены дубликаты, пропущенные значения заменены на `'unknown'`.

## Проверка гипотез

### Сравнение поведения пользователей двух столиц

Первая гипотеза утверждает, что пользователи по-разному слушают музыку в Москве и Санкт-Петербурге. Проверим это предположение по данным о трёх днях недели — понедельнике, среде и пятнице. Для этого:

* Разделим пользователей Москвы и Санкт-Петербурга;
* Сравним, сколько треков послушала каждая группа пользователей в понедельник, среду и пятницу.

Для справки, сначала посмотрим на общие даные по городам и дням

In [18]:
# Подсчёт прослушиваний в каждом городе
df.groupby('city')['city'].count()

city
Moscow              42741
Saint-Petersburg    18512
Name: city, dtype: int64

В Москве прослушиваний больше, чем в Петербурге. Из этого не следует, что московские пользователи чаще слушают музыку. Просто самих пользователей в Москве может быть больше.

In [19]:
# Подсчёт прослушиваний в каждый из трёх дней
df.groupby('day').count()[['user_id']]

,user_id
day,
Friday,21840
Monday,21354
Wednesday,18059


В среднем пользователи из двух городов менее активны по средам. Но картина может измениться, если рассмотреть каждый город в отдельности.

In [20]:
# функция с двумя параметрами: day, city.

def number_tracks(day, city):
    
# В переменной track_list сохраняются те строки таблицы df, для которых 
# значение в столбце 'day' равно параметру day и одновременно значение
# в столбце 'city' равно параметру city 

    track_list = df[df['day'] == day]
    track_list = track_list[track_list['city'] == city]
    
# В переменной track_list_count сохраняется число значений столбца 'user_id',
# рассчитанное методом count() для таблицы track_list.

    track_list_count = track_list['user_id'].count()
    
# Функция возвращает число - значение track_list_count.

    return track_list_count

Вызовите `number_tracks()` шесть раз, меняя значение параметров — так, чтобы получить данные для каждого города в каждый из трёх дней.

In [21]:
for weekday, city in zip(['Monday', 'Wednesday', 'Friday']*2, sorted(['Moscow', 'Saint-Petersburg']*3)):
    print(f'Количество прослушиваний в {city} по {weekday} равно {number_tracks(weekday, city)}')

Количество прослушиваний в Moscow по Monday равно 15740
Количество прослушиваний в Moscow по Wednesday равно 11056
Количество прослушиваний в Moscow по Friday равно 15945
Количество прослушиваний в Saint-Petersburg по Monday равно 5614
Количество прослушиваний в Saint-Petersburg по Wednesday равно 7003
Количество прослушиваний в Saint-Petersburg по Friday равно 5895


Создадим таблицу.

In [22]:
row1 = ['Moscow']
row2 = ['Saint-Petersburg']
for day in ['Monday', 'Wednesday', 'Friday']:
    row1.append(number_tracks(day, row1[0]))
    row2.append(number_tracks(day, row2[0]))
col_names = ['city', 'monday', 'wednesday', 'friday']
calc_data = [row1, row2]
pd.DataFrame(data = calc_data, columns = col_names)

,city,monday,wednesday,friday
0,Moscow,15740,11056,15945
1,Saint-Petersburg,5614,7003,5895


**Выводы**

Данные показывают разницу в поведении пользователей:

- В Москве пик прослушиваний приходится на понедельник и пятницу, а в среду заметен спад.
- В Петербурге, наоборот, больше слушают музыку по средам. Активность в понедельник и пятницу здесь почти в равной мере уступает среде.

Значит, данные говорят в пользу первой гипотезы.

### Музыка в начале и в конце недели

Согласно второй гипотезе, утром в понедельник в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.

In [23]:
# получение таблицы moscow_general 

moscow_general = df[df['city'] == 'Moscow']

In [24]:
# получение таблицы spb_general 

spb_general = df[df['city'] == 'Saint-Petersburg']

Создадим функцию `genre_weekday()` с четырьмя параметрами:
* таблица (датафрейм) с данными,
* день недели,
* начальная временная метка в формате 'hh:mm', 
* последняя временная метка в формате 'hh:mm'.

Функция будет возвращать информацию о топ-10 жанров тех треков, которые прослушивали в указанный день, в промежутке между двумя отметками времени.

In [25]:
# Объявление функции genre_weekday() с параметрами table, day, time1, time2,

def genre_weekday(table, day, time1, time2):

# срез по дню и времени
    genre_df = table[table['day'] == day]
    genre_df = genre_df[genre_df['time'] > time1]
    genre_df = genre_df[genre_df['time'] < time2]
    
# группировка по жанру
    genre_df_count = genre_df.groupby('genre').count()
    
# сортировка по убыванию
    genre_df_sorted = genre_df_count.sort_values(by = 'city', ascending = False)
    
# врозвращаем Series из 10 первых значений 
    return genre_df_sorted.head(10)

Сравним результаты функции `genre_weekday()` для Москвы и Санкт-Петербурга в понедельник утром (с 7:00 до 11:00) и в пятницу вечером (с 17:00 до 23:00):

In [26]:
# вызов функции для утра понедельника в Москве 

genre_weekday(moscow_general, 'Monday', '07:00', '11:00')['user_id'].to_frame()

,user_id
genre,
pop,781
dance,549
electronic,480
rock,474
hiphop,286
ruspop,186
world,181
rusrap,175
alternative,164


In [27]:
# вызов функции для утра понедельника в Петербурге

genre_weekday(spb_general, 'Monday', '07:00', '11:00')['user_id'].to_frame()

,user_id
genre,
pop,218
dance,182
rock,162
electronic,147
hiphop,80
ruspop,64
alternative,58
rusrap,55
jazz,44


In [28]:
# вызов функции для вечера пятницы в Москве

genre_weekday(moscow_general, 'Friday', '17:00', '23:00')['user_id'].to_frame()

,user_id
genre,
pop,713
rock,517
dance,495
electronic,482
hiphop,273
world,208
ruspop,170
alternative,163
classical,163


In [29]:
# вызов функции для вечера пятницы в Петербурге

genre_weekday(spb_general, 'Friday', '17:00', '23:00')['user_id'].to_frame()

,user_id
genre,
pop,256
electronic,216
rock,216
dance,210
hiphop,97
alternative,63
jazz,61
classical,60
rusrap,59


**Выводы**

Если сравнить топ-10 жанров в понедельник утром, можно сделать такие выводы:

1. В Москве и Петербурге слушают похожую музыку. Единственное отличие — в московский рейтинг вошёл жанр “world”, а в петербургский — джаз и классика.

2. В Москве пропущенных значений оказалось так много, что значение `'unknown'` заняло десятое место среди самых популярных жанров. Значит, пропущенные значения занимают существенную долю в данных и угрожают достоверности исследования.

Вечер пятницы не меняет эту картину. Некоторые жанры поднимаются немного выше, другие спускаются, но в целом топ-10 остаётся тем же самым.

Таким образом, вторая гипотеза подтвердилась лишь частично:
* Пользователи слушают похожую музыку в начале недели и в конце.
* Разница между Москвой и Петербургом не слишком выражена. В Москве чаще слушают русскую популярную музыку, в Петербурге — джаз.

Однако пропуски в данных ставят под сомнение этот результат. В Москве их так много, что рейтинг топ-10 мог бы выглядеть иначе, если бы не утерянные  данные о жанрах.

### Жанровые предпочтения в Москве и Петербурге

Гипотеза: Петербург — столица рэпа, музыку этого жанра там слушают чаще, чем в Москве.  А Москва — город контрастов, в котором, тем не менее, преобладает поп-музыка.

Жанры Москвы:

In [30]:
(
moscow_general
    .groupby('genre')['genre']
    .count()
    .sort_values(ascending = False)
    .to_frame()
    .head(10)
)    

,genre
genre,
pop,5892
dance,4435
rock,3965
electronic,3786
hiphop,2096
classical,1616
world,1432
alternative,1379
ruspop,1372


Жанры Петербурга:

In [31]:
(
spb_general
    .groupby('genre')['genre']
    .count()
    .sort_values(ascending = False)
    .to_frame()
    .head(10)
)    

,genre
genre,
pop,2431
dance,1932
rock,1879
electronic,1736
hiphop,960
alternative,649
classical,646
rusrap,564
ruspop,538


**Выводы**

Гипотеза частично подтвердилась:
* Поп-музыка — самый популярный жанр в Москве, как и предполагала гипотеза. Более того, в топ-10 жанров встречается близкий жанр — русская популярная музыка.
* Вопреки ожиданиям, рэп одинаково популярен в Москве и Петербурге. 


## Выводы

Проверено три гипотезы и установлено следующее:

1. День недели по-разному влияет на активность пользователей в Москве и Петербурге. 

Первая гипотеза полностью подтвердилась.

2. Музыкальные предпочтения не сильно меняются в течение недели — будь то Москва или Петербург. Небольшие различия заметны в начале недели, по понедельникам:
* в Москве слушают музыку жанра “world”,
* в Петербурге — джаз и классику.

Таким образом, вторая гипотеза подтвердилась лишь отчасти. Этот результат мог оказаться иным, если бы не пропуски в данных.

3. Во вкусах пользователей Москвы и Петербурга больше общего чем различий. Вопреки ожиданиям, предпочтения жанров в Петербурге напоминают московские, первые пять позиций в ТОП-10 жанров идентичны.

Третья гипотеза не подтвердилась. Если различия в предпочтениях и существуют, на основной массе пользователей они незаметны.